In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [2]:

protease = pd.read_csv('C:\\Users\\Rebe\\Documents\\Python programms\\Capstone_code\\Data\\hiv_protease_inhibitors_data_chemlb.csv')
protease = protease['canonical_smiles']
print(protease.shape)

nrti = pd.read_csv('C:\\Users\\Rebe\\Documents\\Python programms\\Capstone_code\\Data\\nrtis_hiv_chemlb_full.csv')
nrti = nrti['canonical_smiles']
print(nrti.shape)

#Add protease and nrtis data together

smile_3 = pd.concat([protease, nrti], axis=0)
print(smile_3.shape)
smile_3.head()
smile_3.to_csv('smile_3.csv', index=False)

hello = pd.read_csv('smile_3.csv')

hello.head()


(43,)
(9,)
(52,)


,canonical_smiles
0,CC(C)CN(Sc1ccc2c(c1)CCO2)[C@H](CO)CCCCNC(=O)[C...
1,CC(C)CN(Sc1ccc2c(c1)OCCO2)[C@H](CO)CCCCNC(=O)[...
2,CC(C)CN([C@H](CO)CCCCNC(=O)[C@H](Cc1ccccc1Br)N...
3,Cc1c(O)cccc1C(=O)N[C@@H](Cc1ccccc1Br)C(=O)NCCC...
4,CC(C)CN([C@H](CO)CCCCNC(=O)[C@H](Cc1ccccc1Br)N...


In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np

# Load the datasets
protease_path = 'C:\\Users\\Rebe\\Documents\\Python programms\\Capstone_code\\Data\\hiv_protease_inhibitors_data_chemlb.csv'
nrti_path = 'C:\\Users\\Rebe\\Documents\\Python programms\\Capstone_code\\Data\\nrtis_hiv_chemlb_full.csv'

protease_df = pd.read_csv(protease_path)['canonical_smiles']
nrti_df = pd.read_csv(nrti_path)['canonical_smiles']

print(f"Protease dataset shape: {protease_df.shape}")
print(f"NRTI dataset shape: {nrti_df.shape}")

# Generate RDKit Mol objects
protease_mols = [Chem.MolFromSmiles(smiles) for smiles in protease_df if smiles is not None]
nrti_mols = [Chem.MolFromSmiles(smiles) for smiles in nrti_df if smiles is not None]

# Generate fingerprints
protease_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024) for mol in protease_mols]
nrti_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024) for mol in nrti_mols]

# Calculate pairwise Tanimoto similarities
# Note: This can be computationally intensive for large datasets
similarity_matrix = []
for protease_fp in protease_fps:
    row_similarities = [DataStructs.FingerprintSimilarity(protease_fp, nrti_fp) for nrti_fp in nrti_fps]
    similarity_matrix.append(row_similarities)

# Convert similarity matrix to a numpy array for easier analysis
similarity_array = np.array(similarity_matrix)

# Example analysis: Average similarity of each protease inhibitor to all NRTIs
average_similarity_per_protease = similarity_array.mean(axis=1)
print(f"Average Tanimoto similarity per protease inhibitor to all NRTIs: {average_similarity_per_protease.mean()}")

# This is a basic analysis. You might want to explore further, e.g., max or min similarities, or distribution analyses.


Protease dataset shape: (43,)
NRTI dataset shape: (9,)
Average Tanimoto similarity per protease inhibitor to all NRTIs: 0.09415152618379889


In [ ]:
#statistical test

def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(3)

# actives and inactives
  selection = [descriptor, 'type']
  df = df_final[selection]
  protease = df[df.type == 'protease']
  protease = protease[descriptor]

  selection = [descriptor, 'type']
  df = df_final[selection]
  nrti = df[df.type == 'nrti']
  nrti = nrti[descriptor]

# compare samples
  stat, p = mannwhitneyu(protease, nrti)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'
  
  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  #results.to_csv(filename)

  return results

In [ ]:
# modification to create the dataframe for the statistical analysis of all the descriptors


def all_mannwhitney():  
    from numpy.random import seed
    from scipy.stats import mannwhitneyu
    import pandas as pd  # Imported pandas for DataFrame operations

    list_d = ['MW', 'LogP', 'hbd', 'hba', 'psa', 'num_ro5_violations', 'cx_logd', 'np_likeness_score', 'pIC50', 'mw_freebase', 'full_mwt',
              'NumHAcceptors', 'NumHDonors']

    # Initialize results DataFrame
    results = pd.DataFrame(columns=['Descriptor', 'Statistics', 'p', 'alpha', 'Interpretation'])

    # Seed the random number generator
    seed(3)

    for descriptor in list_d:
        selection = [descriptor, 'type']
        df = df_final[selection]
        protease = df[df.type == 'protease'][descriptor]
        nrti = df[df.type == 'nrti'][descriptor]

        # Compare samples
        stat, p = mannwhitneyu(protease, nrti)

        # Interpret results
        alpha = 0.05
        if p > alpha:
            interpretation = 'Same distribution (fail to reject H0)'
        else:
            interpretation = 'Different distribution (reject H0)'

        temp_df = pd.DataFrame({'Descriptor': [descriptor],
                                'Statistics': [stat],
                                'p': [p],
                                'alpha': [alpha],
                                'Interpretation': [interpretation]})

        results = pd.concat([results, temp_df], ignore_index=True)

    return results

In [ ]:
all_mannwhitney()

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from tqdm import tqdm
import os
import re
import sys
import time
import logging
import tempfile
from collections import Counter
from typing import List, Tuple
from tqdm import tqdm

import numpy as np
import pandas as pd

import rdkit
from rdkit import Chem
from rdkit.Chem import MolFromSmiles, MolToSmiles, CanonSmiles

import deepchem as dc
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_validate
from scipy.sparse import csr_matrix, lil_matrix

import atomInSmiles as ais
import json

In [ ]:
# This vocab only focus in characters not in pairs.
class CreateVocab:
    def __init__(self, datasets):
        """
        Initialize the CreateVocab object with multiple datasets.

        :param datasets: List of datasets (strings) to build the vocabulary from.
        """
        self.stoi = {}
        self.itos = {}
        self.build_vocab(datasets)

    def build_vocab(self, datasets):
        """
        Builds the vocabulary from the given datasets.

        :param datasets: List of datasets (strings) to build the vocabulary from.
        """
        all_text = ''.join(datasets)  # Concatenate all datasets
        unique_chars = sorted(set(all_text))  # Extract unique characters

        self.stoi = {char: idx for idx, char in enumerate(unique_chars)}
        self.itos = {idx: char for idx, char in enumerate(unique_chars)}

    def encode(self, text):
        """
        Encodes a given text into a list of indices.

        :param text: String to be encoded.
        :return: List of indices corresponding to the characters in the text.
        """
        return [self.stoi[char] for char in text]

    def decode(self, indices):
        """
        Decodes a list of indices into a string.

        :param indices: List of indices to be decoded.
        :return: Decoded string.
        """
        return ''.join(self.itos[idx] for idx in indices)


def preprocess_smiles(smiles):
    """
    Preprocesses a given SMILES string by adding padding and start and end tokens.

    :param smiles: SMILES string to be preprocessed.
    :return: Preprocessed SMILES string.
    """
    text = [str(i) + "*" for i in smiles]
    text = str(text)[1:-1]
    return text

# Load and process datasets
smile_1 = pd.read_csv('smile_1.csv')['smiles']
smile_2 = pd.read_csv('smile_2.csv')['smiles']
smile_3 = pd.read_csv('smile_3.csv')['canonical_smiles']

processed_smile_1 = preprocess_smiles(smile_1)
processed_smile_2 = preprocess_smiles(smile_2)
processed_smile_3 = preprocess_smiles(smile_3)

# Combine processed datasets
combined_text = processed_smile_1 + processed_smile_2 + processed_smile_3

# Create vocabulary
vocab = CreateVocab([combined_text])

In [ ]:
encoded_text = vocab.encode(text)

# Train and test splits
data = torch.tensor(encoded_text, dtype=torch.long)
n = int(0.9 * len(data))  # 90% for training, rest for validation
train_data = data[:n]
val_data = data[n:]

In [ ]:
#Create section vocab

In [ ]:
#create character vocab 

In [ ]:
encoded_text = vocab.encode(text)

# Train and test splits
data = torch.tensor(encoded_text, dtype=torch.long)
n = int(0.9 * len(data))  # 90% for training, rest for validation
train_data = data[:n]
val_data = data[n:]

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 350
eval_interval = 10
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)
vocab_size = len(vocab.stoi)

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = BigramLanguageModel()

In [ ]:
model_state_path = "2_BWC.pkl"
model.load_state_dict(torch.load(model_state_path))

In [ ]:
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:

smiles_1 = pd.read_csv('smile_3.csv')
#smiles_1
smiles_1 = smiles_1['canonical_smiles']
#smiles_1


text = [str(i) + "*" for i in smiles_1]

text = str(text)[1:-1]
print(text)

In [ ]:
val = vocab.decode(m.generate(context, max_new_tokens=2000)[0].tolist())

In [ ]:
new = "".join(val)
new = new.split('*')

new = [s for s in new if len(s) > 2]

new = [s.replace(' ', '') for s in new]
new

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole


mols = [Chem.MolFromSmiles(smi) for smi in new if Chem.MolFromSmiles(smi) is not None]
Draw.MolsToGridImage(mols, molsPerRow=5, subImgSize=(200, 200), legends=[str(i) for i in range(len(mols))])